In [ ]:
# See http://sbolstandard.org/ontology/ for SBOL ontology information

In [ ]:
import sys

In [ ]:
# Import Interactive Widgets for Jupyter Notebook (https://github.com/jupyter-widgets/ipywidgets)
# See https://ipywidgets.readthedocs.io for documentation on ui widgets
from ipywidgets import widgets

In [ ]:
try:
    from sd2_dictionary.sd2_dictionary_writer import SD2DictionaryWriter
except ImportError:
    # Install the dictionary writer library
    !{sys.executable} -m pip install --user --upgrade git+https://github.com/SD2E/sbol-dictionary-writer
    from sd2_dictionary.sd2_dictionary_writer import SD2DictionaryWriter

In [ ]:
# This may require authorization via google to get access to the google sheet representing the dictionary

dictionaryWriter = SD2DictionaryWriter(console=True)

In [ ]:
def make_filter():
    attrs = dictionaryWriter.dictionary.type_tabs.keys()
    attr_select = widgets.Dropdown(
        options=attrs,
        description='with '
    )
    hbox = widgets.HBox((attr_select, widgets.Label('[specialized]')))
    return hbox

def make_widgets():
    widget_box = widgets.VBox()

    sbh_types = dictionaryWriter.dictionary.type_tabs.keys()
    type_select = widgets.Dropdown(
        options=sbh_types,
        description='Look for '
    )

    add_filter = widgets.Button(
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Add a filter',
        icon='plus'
    )
    
    def add_filter_clicked(b):
        children = widget_box.children[:-1]
        add_btn = (widget_box.children[-1],)
        filter_box = make_filter()
        widget_box.children = children + (filter_box,) + add_btn

    add_filter.on_click(add_filter_clicked)
    widget_box.children = (widgets.HBox((type_select,)), add_filter)
    return widget_box

widget_box = make_widgets()
display(widget_box)

In [ ]:
# Dig the values out of the widgets above

look_for = widget_box.children[0].children[0].value
print('Looking for {}'.format(look_for))
for hbox in widget_box.children[1:-1]:
    attr = hbox.children[0].value
    print('with {}'.format(attr))

# Supporting Code

In [ ]:
try:
    import synbiohub_adapter as sbha
except ImportError:
    # Install synbiohub
    !{sys.executable} -m pip install --user --upgrade git+https://github.com/SD2E/synbiohub_adapter.git
    import synbiohub_adapter as sbha


# Functions and constants in synbiohub_adapter.explore are not
# yet exported, so the package needs to be explicitly imported
import synbiohub_adapter.explore as explore

In [11]:
# Authenticate to SynBioHub

# Adjust these values as needed
SBH_USER = 'scott'
SBH_PASSWORD = 'tiger'

sbh_query = sbha.SynBioHubQuery(sbha.SD2Constants.SD2_SERVER)
sbh_query.login(SBH_USER, SBH_PASSWORD)

# Upward Traversal

In [ ]:
def parent_module_definitions(sbh_query, uri):
    """Find all the module definitions that contain this element. This
    function does not recursively find grandparents, etc. It only goes
    one level up via either a module or functionaComponent predicate.
    """
    # First, gather module definitions found via an intervening "Module" node
    modules = explore.subjects_for(sbh_query, explore.SBOL_DEFINITION, uri)
    module_defs = [explore.subjects_for(sbh_query, explore.SBOL_MODULE, m) for m in modules]
    result = set(md for md_list in module_defs for md in md_list)
    # Second, gather module definitions found via an intervening "Functional Component" node
    functional_components = explore.subjects_for(sbh_query, explore.SBOL_DEFINITION, uri)
    module_defs = [explore.subjects_for(sbh_query, explore.SBOL_FUNCTIONAL_COMPONENT, fc) for fc in functional_components]
    result.update(set(md for md_list in module_defs for md in md_list))
    return result

In [ ]:
mds = parent_module_definitions(sbh_query, 'https://hub.sd2e.org/user/sd2e/design/UWBF_5783/1')
for md in mds:
    display(md)
    display(parent_module_definitions(sbh_query, md))

In [ ]:
# Test for genetic construct
mds = parent_module_definitions(sbh_query, 'https://hub.sd2e.org/user/sd2e/design/YG_plasmid_002/1')
for md in mds:
    display(md)
    display(parent_module_definitions(sbh_query, md))

# Downward traversal

In [ ]:
import logging
logging.root.level = logging.INFO

In [ ]:
# A wrapper with a better function name than "o_query"
def objects_for(sbh_query, subj, pred):
    return explore.o_query(sbh_query, subj, pred)

# Introduce the function above to the explore module
explore.objects_for = objects_for

def child_module_definitions(sbh_query, uri):
    """Find all children that are module definitions
    """
    # Gather module definitions found via an intervening "Module" node
    modules = explore.objects_for(sbh_query, uri, explore.SBOL_MODULE)
    
    module_defs = [explore.objects_for(sbh_query, m, explore.SBOL_DEFINITION) for m in modules]
    result = set(md for md_list in module_defs for md in md_list)
    return result

def child_component_definitions(sbh_query, uri):
    """Find all children that are component definitions.
    """
    # Gather component definitions found via an intervening "FunctionalComponent" node
    # uri --functionalComponent--> FunctionComponent --definition--> ComponentDefinition
    fcs = explore.objects_for(sbh_query, uri, explore.SBOL_FUNCTIONAL_COMPONENT)
    comp_defs = [explore.objects_for(sbh_query, fc, explore.SBOL_DEFINITION) for fc in fcs]
    result = set(cd for cd_list in comp_defs for cd in cd_list)
    return result


In [ ]:
uri = 'https://hub.sd2e.org/user/sd2e/experiment_test/experiment0x2etranscriptic0x2er1c7cpvfzqprk_r1c7fbvba55db_group_2/1'
uri = 'https://hub.sd2e.org/user/sd2e/experiment_test/experiment0x2euw_biofab0x2e10843_group_2/1'
children = child_module_definitions(sbh_query, uri)
children

In [ ]:
for child in children:
    print(child)
    print('Child Module Definitions')
    print(child_module_definitions(sbh_query, child))
    print('Child Component Definitions')
    child_cds = child_component_definitions(sbh_query, child)
    print(child_cds)
    for cd in child_cds:
        display(explore.subject_info(sbh_query, cd))


In [ ]:
strain_uri = 'https://hub.sd2e.org/user/sd2e/design/UWBF_5783/1'
strain_uri = 'https://hub.sd2e.org/user/sd2e/design/UWBF_16967/1'
strain_uri = 'https://hub.sd2e.org/user/sd2e/design/NEB_10_beta_pAN3928_pAN4036_Larabinose_aTc_IPTG_system/1'
strain_uri = 'https://hub.sd2e.org/user/sd2e/design/CY9680x2DCY662int/1'
explore.subject_info(sbh_query, strain_uri)